## First Hypotheses:

O próximo município de atuação deve estar inserido em um contexto similar aos atuais.

### 1) Importing Libraries

In [112]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import warnings

# Ou suprimir todos os warnings
warnings.filterwarnings("ignore")

### 2) Data Cleaning and Loading:

In [3]:
codigos_ne = list(range(21, 30))
lista_municipios_modelo = ["MAURITI", "INAJÁ", "SÃO JOSÉ DA TAPERA", "BUÍQUE"]

#### 2.1) Dados dos Municípios

In [4]:
df_municipios_raw_um = pd.read_excel("..\Dados\Municipios\lista-1262municipios-semiarido-2017.xlsx", header=2)
df_municipios_raw_dois = pd.read_csv("..\Dados\Municipios\municipios.csv")

In [5]:
df_municipios_um = df_municipios_raw_um.copy()
df_municipios_dois = df_municipios_raw_dois.copy()

In [6]:
display(df_municipios_um.info())
display(df_municipios_dois.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1262 entries, 0 to 1261
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   #                          1262 non-null   int64 
 1   Cod IBGE                   1262 non-null   int64 
 2   UF                         1262 non-null   object
 3   Município                  1262 non-null   object
 4   População Estimada - 2017  1262 non-null   object
dtypes: int64(2), object(3)
memory usage: 49.4+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   codigo_ibge  5570 non-null   int64  
 1   nome         5570 non-null   object 
 2   latitude     5570 non-null   float64
 3   longitude    5570 non-null   float64
 4   codigo_uf    5570 non-null   int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 217.7+ KB


None

In [8]:
df_municipios_um.drop(columns=["#"], inplace=True)

In [9]:
df_municipios_dois.columns

Index(['codigo_ibge', 'nome', 'latitude', 'longitude', 'codigo_uf'], dtype='object')

In [10]:
df_municipios = pd.merge(df_municipios_um, df_municipios_dois[["codigo_ibge", "codigo_uf", "latitude", "longitude"]], 
                         how="inner", left_on="Cod IBGE", right_on="codigo_ibge")

df_municipios.drop(columns=["Cod IBGE"], inplace=True)

In [12]:
municipios_ne = df_municipios[df_municipios["codigo_uf"].isin(codigos_ne)]
codigos_municipios_modelos = municipios_ne["codigo_ibge"].values

In [44]:
municipios_ne.head()

,UF,Município,População Estimada - 2017,codigo_ibge,codigo_uf,latitude,longitude
0,AL,ÁGUA BRANCA,20467,2700102,27,-9.26200,-37.9380
1,AL,ARAPIRACA,234185,2700300,27,-9.75487,-36.6615
2,AL,BATALHA,18757,2700706,27,-9.67420,-37.1330
3,AL,BELO MONTE,6797,2700904,27,-9.82272,-37.2770
4,AL,CACIMBINHAS,10897,2701209,27,-9.40121,-36.9911


#### 2.2) Registros Administrativos:

Buscar informações sobre educação dentro do banco de dados de Resgistros administrativos que façam sentido com o que a ONG, Amigos do bem, busque dentro de um modelo de município:

- Taxa de evasão do ensino fundamental (TTREVA_EF_TOTAL) (sem dados)
- Taxa de evasão do ensino médio (TTREVA_EM_TOTAL) (sem dados)
- Taxa de evasão do ensino médio em Escolas públicas (TTREVA_EM_PUB). (sem dados)
- Taxa de evasão do ensino fundamental em Escolas públicas (TTREVA_EF_PUB). (sem dados)
- Percentual de matrículas da rede pública no ensino fundamental (PMATPUB_EF).
- Percentual de matrículas da rede pública no ensino médio (PMATPUB_EM)

In [41]:
df_adm_total_raw = pd.read_excel(r"..\Dados\Registros Administrativos\dados_registros_administrativos_total_2012_2017.xlsx", sheet_name="MUNICÍPIO")

In [74]:
df_adm_total = df_adm_total_raw.copy()

In [83]:
lista_colunas_uteis_adm = ["ANO","IBGE7", "NOME", "TTREVA_EF_TOTAL", "TTREVA_EM_TOTAL", "TTREVA_EF_PUB", 
                           "TTREVA_EM_PUB", "PMATPUB_EF", "PMATPUB_EM"]

In [84]:
df_util_adm = df_adm_total[lista_colunas_uteis_adm][df_adm_total["ANO"] == 2017]

In [85]:
df_util_adm.drop(columns="ANO", inplace=True)

In [86]:
df_util_adm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5570 entries, 22280 to 27849
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   IBGE7            5570 non-null   int64  
 1   NOME             5570 non-null   object 
 2   TTREVA_EF_TOTAL  0 non-null      float64
 3   TTREVA_EM_TOTAL  0 non-null      float64
 4   TTREVA_EF_PUB    0 non-null      float64
 5   TTREVA_EM_PUB    0 non-null      float64
 6   PMATPUB_EF       5570 non-null   float64
 7   PMATPUB_EM       5561 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 391.6+ KB


In [88]:
df_util_adm = df_util_adm[["IBGE7", "PMATPUB_EF", "PMATPUB_EM"]]

,IBGE7,PMATPUB_EF,PMATPUB_EM
22280,1100015,100.00,100.00
22281,1100023,89.42,95.84
22282,1100031,100.00,100.00
22283,1100049,82.46,80.47
22284,1100056,96.31,97.47


In [89]:
df_util_adm.isna().sum()

IBGE7         0
PMATPUB_EF    0
PMATPUB_EM    9
dtype: int64

In [91]:
imputer = SimpleImputer(strategy="mean")
df_util_adm["PMATPUB_EM"] = imputer.fit_transform(df_util_adm[["PMATPUB_EM"]])

In [93]:
df_util_adm.set_index("IBGE7", inplace=True)

In [94]:
df_util_adm[["PMATPUB_EF", "PMATPUB_EM"]] = df_util_adm[["PMATPUB_EF", "PMATPUB_EM"]] / 100 

In [96]:
# Tabela final para registros administrativos:
df_util_adm.head()

,PMATPUB_EF,PMATPUB_EM
IBGE7,,
1100015,1.0000,1.0000
1100023,0.8942,0.9584
1100031,1.0000,1.0000
1100049,0.8246,0.8047
1100056,0.9631,0.9747


#### 2.3) Dados Educacionais e Censo 2020:

In [1]:
edu_raw_2020 = pd.read_csv(r"..\Dados\Q EDU\Dados_QEdu_Analitico_2020.csv", delimiter=";")
censo_raw_2020 = pd.read_excel(r"..\Dados\Censo\estimativa_dou_2020.xls", header=1, sheet_name="Municípios", dtype={"COD. MUNIC": str})

NameError: name 'pd' is not defined

In [311]:
edu_2020 = edu_raw_2020.copy()
censo_2020 = censo_raw_2020.copy()

In [312]:
edu_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1795 entries, 0 to 1794
Data columns (total 76 columns):
 #   Column                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                  --------------  -----  
 0   COD Municipio                                                                                                           1795 non-null   object 
 1   Cidade                                                                                                                  1794 non-null   object 
 2   COD UF                                                                                                                  1794 non-null   float64
 3   UF                                                                                                                      1794 non-null   object 

In [313]:
edu_2020_nordeste = edu_2020[edu_2020["COD UF"].isin(codigos_ne)]

In [314]:
edu_2020_nordeste["Total de escolas estaduais"] = edu_2020_nordeste["Total de escolas estaduais"].str.replace(",", ".")
edu_2020_nordeste["Total de escolas municipais"] = edu_2020_nordeste["Total de escolas municipais"].str.replace(",", ".")

In [315]:
edu_2020_nordeste[[ 
                "Total de escolas estaduais", 
                "Total de escolas municipais"]] = edu_2020_nordeste[[ 
                                                                    "Total de escolas estaduais", 
                                                                    "Total de escolas municipais"]].astype('float').astype('Int64')

In [316]:
imputer = SimpleImputer(strategy="mean")
dados_imputados = imputer.fit_transform(edu_2020_nordeste[[ 
                                                           "Total de escolas estaduais", 
                                                           "Total de escolas municipais"]])

edu_2020_nordeste[["Total de escolas estaduais", "Total de escolas municipais"]] = dados_imputados.round()

##### 2.7.1) Número de escolas municipais e estaduais em um município

In [317]:
edu_2020_nordeste["Total_escolas_publicas_mun_est"] = edu_2020_nordeste['Total de escolas estaduais'] \
    + edu_2020_nordeste['Total de escolas municipais']

In [318]:
df_edu_2020 = edu_2020_nordeste[["COD Municipio", "Total_escolas_publicas_mun_est"]]

df_edu_2020.set_index("COD Municipio", inplace=True)

In [319]:
df_edu_2020.head()

,Total_escolas_publicas_mun_est
COD Municipio,
2507507,289.0
2207702,108.0
2412005,65.0
2302800,88.0
2110104,74.0


##### 2.7.2) Número de matrículas por população jovem:

In [320]:
total_matriculas =  edu_2020_nordeste[["COD Municipio", "Total de matrículas"]]
total_matriculas.set_index("COD Municipio", inplace=True)

In [321]:
total_matriculas.isna().sum()

Total de matrículas    0
dtype: int64

In [322]:
df_edu_2020 = df_edu_2020.merge(total_matriculas, how="inner", left_index=True, right_index=True)

In [323]:
df_edu_2020.head()

,Total_escolas_publicas_mun_est,Total de matrículas
COD Municipio,,
2507507,289.0,169336.0
2207702,108.0,38350.0
2412005,65.0,24709.0
2302800,88.0,20928.0
2110104,74.0,8861.0


In [324]:
censo_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5586 entries, 0 to 5585
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   UF                  5584 non-null   object 
 1   COD. UF             5570 non-null   float64
 2   COD. MUNIC          5570 non-null   object 
 3   NOME DO MUNICÍPIO   5570 non-null   object 
 4   POPULAÇÃO ESTIMADA  5570 non-null   object 
dtypes: float64(1), object(4)
memory usage: 218.3+ KB


In [325]:
censo_2020_ne = censo_2020[censo_2020["COD. UF"].isin(codigos_ne)]

In [326]:
censo_2020_ne

,UF,COD. UF,COD. MUNIC,NOME DO MUNICÍPIO,POPULAÇÃO ESTIMADA
450,MA,21.0,00055,Açailândia,113121
451,MA,21.0,00105,Afonso Cunha,6578
452,MA,21.0,00154,Água Doce do Maranhão,12652
453,MA,21.0,00204,Alcântara,22112
454,MA,21.0,00303,Aldeias Altas,26757
...,...,...,...,...,...
2239,BA,29.0,33307,Vitória da Conquista,341128
2240,BA,29.0,33406,Wagner,9344
2241,BA,29.0,33455,Wanderley,12180
2242,BA,29.0,33505,Wenceslau Guimarães,20978


In [327]:
censo_2020_ne["COD. UF"] = censo_2020_ne["COD. UF"].astype(int).astype(str) 

In [328]:
censo_2020_ne

,UF,COD. UF,COD. MUNIC,NOME DO MUNICÍPIO,POPULAÇÃO ESTIMADA
450,MA,21,00055,Açailândia,113121
451,MA,21,00105,Afonso Cunha,6578
452,MA,21,00154,Água Doce do Maranhão,12652
453,MA,21,00204,Alcântara,22112
454,MA,21,00303,Aldeias Altas,26757
...,...,...,...,...,...
2239,BA,29,33307,Vitória da Conquista,341128
2240,BA,29,33406,Wagner,9344
2241,BA,29,33455,Wanderley,12180
2242,BA,29,33505,Wenceslau Guimarães,20978


In [329]:
censo_2020_ne["CODIGO_MUNIC"] = censo_2020_ne["COD. UF"] +  censo_2020_ne["COD. MUNIC"]

In [330]:
censo_2020_ne.drop(columns=["COD. UF", "COD. MUNIC", "NOME DO MUNICÍPIO", "UF"], inplace=True)

In [331]:
censo_2020_ne.set_index("CODIGO_MUNIC", inplace=True)

In [332]:
censo_2020_ne

,POPULAÇÃO ESTIMADA
CODIGO_MUNIC,
2100055,113121
2100105,6578
2100154,12652
2100204,22112
2100303,26757
...,...
2933307,341128
2933406,9344
2933455,12180


In [333]:
resultado = censo_2020_ne["POPULAÇÃO ESTIMADA"].str.split("(").to_frame()

In [334]:
index = resultado.dropna().index
valores = [i[0] for i in resultado.dropna()["POPULAÇÃO ESTIMADA"]]

In [335]:
valores_serie = pd.Series(valores, index=index).str.strip()
censo_2020_ne["POPULAÇÃO ESTIMADA"].loc[index] = valores_serie.values

In [336]:
censo_2020_ne["POPULAÇÃO ESTIMADA"] = censo_2020_ne["POPULAÇÃO ESTIMADA"].astype(int)

In [337]:
# Juntando o dataset de educação com a população:
df_edu_2020 = df_edu_2020.merge(censo_2020_ne, how="inner", left_index=True, right_index=True)

In [339]:
df_edu_2020.index.name = "COD_MUNIC"

In [340]:
df_edu_2020

,Total_escolas_publicas_mun_est,Total de matrículas,POPULAÇÃO ESTIMADA
COD_MUNIC,,,
2507507,289.0,169336.0,817511
2207702,108.0,38350.0,153482
2412005,65.0,24709.0,103672
2302800,88.0,20928.0,77244
2110104,74.0,8861.0,25764
...,...,...,...
2917003,60.0,11398.0,36116
2103505,58.0,17674.0,41312
2402006,46.0,15571.0,68343


In [345]:
df_edu_2020["TOT_ESC_POR_POP"] = df_edu_2020["Total_escolas_publicas_mun_est"]/ df_edu_2020["POPULAÇÃO ESTIMADA"]
df_edu_2020["TOT_MAT_POR_POP"] = df_edu_2020["Total de matrículas"]/ df_edu_2020["POPULAÇÃO ESTIMADA"]

In [346]:
df_edu_2020

,Total_escolas_publicas_mun_est,Total de matrículas,POPULAÇÃO ESTIMADA,TOT_ESC_POR_POP,TOT_MAT_POR_POP
COD_MUNIC,,,,,
2507507,289.0,169336.0,817511,0.000354,0.207136
2207702,108.0,38350.0,153482,0.000704,0.249866
2412005,65.0,24709.0,103672,0.000627,0.238338
2302800,88.0,20928.0,77244,0.001139,0.270934
2110104,74.0,8861.0,25764,0.002872,0.343930
...,...,...,...,...,...
2917003,60.0,11398.0,36116,0.001661,0.315594
2103505,58.0,17674.0,41312,0.001404,0.427818
2402006,46.0,15571.0,68343,0.000673,0.227836


In [349]:
df_edu_2020.index = df_edu_2020.index.astype(int)

Dataframe com informações de educação final:

In [354]:
df_edu_2020 = df_edu_2020.merge(df_util_adm, how="inner", left_index=True, right_index=True)

In [355]:
df_edu_2020_final = df_edu_2020.drop(columns=["POPULAÇÃO ESTIMADA", "Total_escolas_publicas_mun_est", "Total de matrículas"])

In [427]:
df_edu_2020_final.drop(columns=['nome'], inplace=True)

In [428]:
df_edu_2020_final.to_csv(r"..\Dados\Q EDU\dados_edu_2020.csv")

#### 3) Tabela final (Educação):

In [430]:
tabela_final = pd.read_csv(r"..\Dados\Tabela_final\tabela_final.csv")

In [431]:
tabela_final = tabela_final[tabela_final["UF"] != "MG"]

In [432]:
tabela_final.set_index("IBGE7", inplace=True)

In [433]:
tabela_final = tabela_final.merge(df_edu_2020_final, how="inner", left_index=True, right_index=True)
